In [1]:
import pandas as pd # require pandas 1.5.3 
import numpy as np
import decouple
import requests
import json
import warnings
import datetime as dt
import os
import re
warnings.filterwarnings("ignore")

import reddit_requests as rr


In [2]:
config = decouple.AutoConfig(' ')
key = config('APIKEY')
pub = config('PUBLICKEY')
user = config('USERNAME')
pw = config('PW')

In [3]:
# use 'kind' key t1_ = comment, t2=acct, t3=link, t4=message, t5=sub, t6=award
auth = requests.auth.HTTPBasicAuth(pub, key)
data = {
    'grant_type': 'password',
    'username': user,
    'password': pw
}

headers = {'User-Agent': 'MYAPI/0.0.1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}


In [ ]:
### scrape lakers stats ###
# historical standings https://www.basketball-reference.com/friv/standings.fcgi?month=10&day=18&year=2022&lg_id=NBA
# current roster https://www.basketball-reference.com/teams/LAL/2023.html#all_roster
# win streak https://www.basketball-reference.com/teams/LAL/2023_games.html
# would be pretty easy to manually do roster changes so not too worried ab that
# otherwise just need game logs for each player and I can code up function to give stats up to that date

In [117]:
### Use pushshift directly ###
subreddit = 'lakers'
start_date = dt.datetime(2022, 10, 15)
end_date = dt.datetime(2022, 10, 19)
start = int(start_date.timestamp())
end = int(end_date.timestamp())

api_query = 'https://api.pushshift.io/reddit/submission/search/' \
            + '?subreddit={}&limit=200&after={}&before={}'.format(subreddit, start, end)

r = requests.get(api_query)
json= r.json()
df = pd.DataFrame(json['data'])


In [118]:
df.shape

(128, 99)

In [158]:
def get_more_posts(start, end, subreddit='lakers', limit=50):
    """ Get reddit posts back to the passed start_date.  
        
        Parameters
        
        -start/end must be in YYYY-MM-DD string format
    
    """
    start = int(start.timestamp())
    end = int(end.timestamp())
    
    api_query = 'https://api.pushshift.io/reddit/submission/search/' \
                + '?subreddit={}&limit={}&after={}&before={}'.format(subreddit, limit, start, end)
    
    try:
        r = requests.get(api_query)
        json= r.json()
        df = pd.DataFrame(json['data'])
    
        df = df[['utc_datetime_str', 'id', 'title', 'author', 'selftext', 'upvote_ratio']]
        print("Successfully pulled data")
        return df
    except:
        print("Upload failed")



In [10]:
# run through a hacky for loop
base = dt.datetime(2023, 4, 15)
date_list = [base - dt.timedelta(days=x) for x in range(186)]

bad_dates = []
for date in date_list:
    start = date
    end = start + dt.timedelta(days=1)
    df = get_more_posts(start, end)
    try:
        start_string = start.strftime('%Y_%m_%d')
        df.to_csv('data/r_lakers_{}'.format(start_string))
        print("Successfully uploaded data for {}".format(start_string))
    except:
        print("Failed to Upload Data for {}".format(start_string))
        bad_dates.append(start_string.replace('_', '-'))


Successfully pulled data
Successfully uploaded data for 2022_10_19
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2022_10_22
Successfully pulled data
Successfully uploaded data for 2022_10_23
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2022_10_27
Successfully pulled data
Successfully uploaded data for 2022_10_28
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2022_10_30
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2022_11_01
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2022_11_04
Successfully pulled data
Successfully uploaded data for 2022_11_05
Successfully pulled data
Successfully uploaded data for 2022_11_06
Upload

Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2023_03_22
Successfully pulled data
Successfully uploaded data for 2023_03_23
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2023_03_25
Successfully pulled data
Successfully uploaded data for 2023_03_26
Successfully pulled data
Successfully uploaded data for 2023_03_27
Successfully pulled data
Successfully uploaded data for 2023_03_28
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2023_03_31
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2023_04_04
Upload failed
Failed to Upload Data
Successfully pulled data
Successfully uploaded data for 2023_04_06
Successfully pulled data
Successfully

In [157]:
# check for min files that need to be re-uploaded
def check_for_max_posts(limit = 50):
    directory = '/Users/dylanjorling/NBASA_reddit/data/daily_posts'
    files = os.listdir(directory)
    files = [x for x in files if len(re.findall(r"r_lakers", x)) == 1]
    max_post_files = []
    for file in files:
        try:
            x = pd.read_csv('data/daily_posts/' + file)
            if x.shape[0] == limit:
                max_post_files.append(file)
        except:
            continue
    max_post_files_date = [file[9:] for file in max_post_files]
    max_post_files_date = [file.replace('_', '-') for file in max_post_files_date]
    
    return max_post_files_date

In [42]:
base = dt.datetime(2023, 4, 12)
date_list = [base - dt.timedelta(days=x) for x in range(186)]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]
file_list = [file.replace('_', '-')[9:] for file in files]
file_list.remove('')

new_date_list = [x for x in date_list if x not in file_list]
print(len(new_date_list))
new_date_list.append(min_files_date)
print(len(new_date_list))

78
79


In [ ]:
start = dt.datetime(2023, 4, 13) #re-do: 
end = dt.datetime(2023, 4, 14)
df2 = get_more_posts(start, end, subreddit='lakers', limit=150)
df2.to_csv('/Users/dylanjorling/NBASA_reddit/data/r_lakers_2023_04_13')
x = pd.read_csv('data/r_lakers_2023_04_13')
x.shape[0]

In [192]:
### concat post data into one df and save ###
directory = '/Users/dylanjorling/NBASA_reddit/data/daily_posts/'
files = os.listdir(directory)
files = [x for x in files if len(re.findall(r"r_lakers", x)) == 1]

df_list = []
for file in files:
    df_list.append(pd.read_csv('data/daily_posts/' + file, index_col=0))
    
df_full = pd.concat(df_list, ignore_index=True)
df_full.reset_index()
df_full=df_full.rename(columns = {'utc_datetime_str':'datetime'})
df_full['datetime'] = pd.to_datetime(df_full['datetime'])
#df_full['date'] = df_full['datetime'].dt.date
df_full = df_full[df_full['datetime'] >= '2022-10-13']
df_full = df_full.sort_values('datetime')
df_full.set_index('datetime', inplace=True)
df_full.reset_index(inplace=True)
df_full.to_csv('data/r_lakers_full_posts.csv')

In [6]:
### save entire comment history with a new csv each month ###
df = pd.read_csv('data/r_lakers_full_posts.csv', index_col=0, parse_dates=['datetime'])
df['month'] = df['datetime'].dt.month_name()
for month in set(df.month):
    df_int = df[df['month'] == month]
    for
    


In [ ]:
### save entire comment history with a new csv each month ###
df = pd.read_csv('data/r_lakers_full_posts.csv', index_col=0, parse_dates=['datetime'])
df['month'] = df['datetime'].dt.month_name()

# iterate through unique months
bad_ids = []
for month in set(df['month']):
    # create monthly dfs
    df_int = df[df['month'] == month]
    
    # define empty df to store comments
    columns = ['datetime', 'tid', 'id', 'author', 'pid', 'body', 'upvotes', 'downvotes']
    dfc_month = pd.DataFrame(columns=columns)
    
    # iterate through each row and append to month
    for i, row in df_int.iterrows():
        try:
            dfc = rr.get_all_comments(tid=row.id,
                                      date=row.datetime,
                                      subreddit='lakers')
            dfc_month = pd.concat([dfc_month, dfc])
            print("tid: {} date: {} successfully loaded".format(row.id, row.datetime))
        except:
            print("tid: {} date: {} failed to load".format(row.id, row.datetime))
            bad_ids.append(row.id)
        
        
    dfc_month.to_csv('data/monthly_comment_hist/{}_comments.csv'.format(month))

tid: yit3tc date: 2022-11-01 00:01:10 successfully loaded
tid: yitb4e date: 2022-11-01 00:10:39 successfully loaded
tid: yitvym date: 2022-11-01 00:36:29 successfully loaded
tid: yiu7jq date: 2022-11-01 00:51:20 successfully loaded
tid: yiu84m date: 2022-11-01 00:52:04 successfully loaded
tid: yivu4g date: 2022-11-01 02:06:37 successfully loaded
tid: yivuuu date: 2022-11-01 02:07:37 successfully loaded
tid: yiwkri date: 2022-11-01 02:42:26 successfully loaded
tid: yiwmyg date: 2022-11-01 02:45:23 failed to load
tid: yiwt16 date: 2022-11-01 02:53:31 successfully loaded
tid: yix2z1 date: 2022-11-01 03:06:54 successfully loaded
tid: yixog4 date: 2022-11-01 03:35:50 successfully loaded
tid: yiz15r date: 2022-11-01 04:45:12 successfully loaded
tid: yiz4ek date: 2022-11-01 04:50:18 successfully loaded
tid: yiz7wd date: 2022-11-01 04:55:41 successfully loaded
tid: yizi55 date: 2022-11-01 05:11:02 successfully loaded
tid: yizouu date: 2022-11-01 05:21:32 successfully loaded
tid: yizyzo date: 2

tid: ykt3ai date: 2022-11-03 05:17:42 successfully loaded
tid: ykt3q0 date: 2022-11-03 05:18:20 successfully loaded
tid: ykt54x date: 2022-11-03 05:20:39 successfully loaded
tid: ykt6yb date: 2022-11-03 05:23:31 successfully loaded
tid: ykt7w3 date: 2022-11-03 05:25:01 failed to load
tid: ykt8c6 date: 2022-11-03 05:25:43 successfully loaded
tid: ykt8pk date: 2022-11-03 05:26:16 successfully loaded
tid: ykt9e3 date: 2022-11-03 05:27:25 failed to load
tid: yktc0a date: 2022-11-03 05:31:42 successfully loaded
tid: yktci5 date: 2022-11-03 05:32:28 successfully loaded
tid: ykteox date: 2022-11-03 05:36:07 successfully loaded
tid: yktfao date: 2022-11-03 05:37:11 successfully loaded
tid: yktfmj date: 2022-11-03 05:37:47 successfully loaded
tid: yktg0a date: 2022-11-03 05:38:24 successfully loaded
tid: yktg7i date: 2022-11-03 05:38:46 successfully loaded
tid: yktglw date: 2022-11-03 05:39:25 successfully loaded
tid: yktij4 date: 2022-11-03 05:42:33 successfully loaded
tid: yktjo6 date: 2022-1

tid: ylzrwy date: 2022-11-04 13:57:56 successfully loaded
tid: ylztt5 date: 2022-11-04 14:00:08 failed to load
tid: ym1t35 date: 2022-11-04 15:21:53 failed to load
tid: ym1uow date: 2022-11-04 15:23:40 successfully loaded
tid: ym2l40 date: 2022-11-04 15:53:26 successfully loaded
tid: ym2nlr date: 2022-11-04 15:56:21 successfully loaded
tid: ym3gsp date: 2022-11-04 16:28:46 successfully loaded
tid: ym3jo2 date: 2022-11-04 16:31:42 successfully loaded
tid: ym3lpo date: 2022-11-04 16:34:02 successfully loaded
tid: ym41wq date: 2022-11-04 16:52:44 failed to load
tid: ym46k9 date: 2022-11-04 16:57:53 successfully loaded
tid: ym4b11 date: 2022-11-04 17:02:34 successfully loaded
tid: ym4y3u date: 2022-11-04 17:28:26 failed to load
tid: ym5h8m date: 2022-11-04 17:49:42 successfully loaded
tid: ym6bed date: 2022-11-04 18:22:19 successfully loaded
tid: ym6f47 date: 2022-11-04 18:26:26 failed to load
tid: ym6xe2 date: 2022-11-04 18:46:52 successfully loaded
tid: ym7b7a date: 2022-11-04 19:02:19 f

tid: ynzvus date: 2022-11-06 19:24:03 successfully loaded
tid: yo02q0 date: 2022-11-06 19:30:20 failed to load
tid: yo0brd date: 2022-11-06 19:38:42 successfully loaded
tid: yo0zcb date: 2022-11-06 20:03:47 successfully loaded
tid: yo13iu date: 2022-11-06 20:07:56 successfully loaded
tid: yo32j4 date: 2022-11-06 21:16:43 successfully loaded
tid: yo3ptg date: 2022-11-06 21:39:30 successfully loaded
tid: yo3yq7 date: 2022-11-06 21:49:43 failed to load
tid: yo40mu date: 2022-11-06 21:51:50 successfully loaded
tid: yo4etw date: 2022-11-06 22:07:45 failed to load
tid: yo4jss date: 2022-11-06 22:13:21 failed to load
tid: yo4maw date: 2022-11-06 22:16:07 successfully loaded
tid: yo55ar date: 2022-11-06 22:38:42 successfully loaded
tid: yo5eea date: 2022-11-06 22:49:38 failed to load
tid: yo5ig0 date: 2022-11-06 22:54:31 failed to load
tid: yo5jy0 date: 2022-11-06 22:56:23 successfully loaded
tid: yo5lj9 date: 2022-11-06 22:58:21 successfully loaded
tid: yo5mra date: 2022-11-06 22:59:53 succes

tid: yp9xin date: 2022-11-08 03:31:10 successfully loaded
tid: yp9z0l date: 2022-11-08 03:33:06 failed to load
tid: ypails date: 2022-11-08 03:59:57 successfully loaded
tid: ypaj33 date: 2022-11-08 04:00:21 successfully loaded
tid: ypalmz date: 2022-11-08 04:03:48 successfully loaded
tid: ypb0d1 date: 2022-11-08 04:23:50 successfully loaded
tid: ypb5mn date: 2022-11-08 04:31:11 failed to load
tid: ypb91w date: 2022-11-08 04:35:58 successfully loaded
tid: ypbamh date: 2022-11-08 04:38:17 failed to load
tid: ypbkdv date: 2022-11-08 04:52:31 successfully loaded
tid: ypbl2l date: 2022-11-08 04:53:36 successfully loaded
tid: ypbpsj date: 2022-11-08 05:00:21 failed to load
tid: ypc2mf date: 2022-11-08 05:18:37 successfully loaded
tid: ypc5bx date: 2022-11-08 05:22:37 successfully loaded
tid: ypc7if date: 2022-11-08 05:25:57 successfully loaded
tid: ypc8i2 date: 2022-11-08 05:27:29 failed to load
tid: ypc918 date: 2022-11-08 05:28:17 failed to load
tid: ypc91f date: 2022-11-08 05:28:18 failed

...

tid: ysyycb date: 2022-11-12 06:50:50 successfully loaded
tid: ysz4ok date: 2022-11-12 06:59:32 successfully loaded
tid: ysz62d date: 2022-11-12 07:01:06 failed to load
tid: yszc6i date: 2022-11-12 07:09:23 successfully loaded
tid: yszl0i date: 2022-11-12 07:21:19 successfully loaded
tid: yszppl date: 2022-11-12 07:27:58 successfully loaded
tid: yszqk2 date: 2022-11-12 07:29:13 successfully loaded
tid: yszr55 date: 2022-11-12 07:30:05 successfully loaded
tid: yszsew date: 2022-11-12 07:31:35 successfully loaded
tid: yszsi5 date: 2022-11-12 07:31:43 successfully loaded
tid: yszsj3 date: 2022-11-12 07:31:45 failed to load
tid: yszu5m date: 2022-11-12 07:34:04 successfully loaded
tid: yt0ab5 date: 2022-11-12 07:56:47 successfully loaded
tid: yt0ozq date: 2022-11-12 08:16:45 successfully loaded
tid: yt1533 date: 2022-11-12 08:39:49 successfully loaded
tid: yt16vt date: 2022-11-12 08:42:18 successfully loaded
tid: yt1d1w date: 2022-11-12 08:51:00 failed to load
tid: yt1wmd date: 2022-11-12 

tid: yuoouz date: 2022-11-14 04:22:20 successfully loaded
tid: yuoq1u date: 2022-11-14 04:24:07 successfully loaded
tid: yuot6g date: 2022-11-14 04:28:47 successfully loaded
tid: yuoud6 date: 2022-11-14 04:30:23 failed to load
tid: yuoumz date: 2022-11-14 04:30:48 successfully loaded
tid: yuoxr6 date: 2022-11-14 04:35:12 successfully loaded
tid: yup5rs date: 2022-11-14 04:47:07 successfully loaded
tid: yup67c date: 2022-11-14 04:47:51 failed to load
tid: yupbv4 date: 2022-11-14 04:56:36 successfully loaded
tid: yupcaz date: 2022-11-14 04:57:17 failed to load
tid: yupcr6 date: 2022-11-14 04:58:02 successfully loaded
tid: yupd1s date: 2022-11-14 04:58:32 successfully loaded
tid: yupdeh date: 2022-11-14 04:59:05 failed to load
tid: yupdv7 date: 2022-11-14 04:59:49 successfully loaded
tid: yupfpa date: 2022-11-14 05:01:58 successfully loaded
tid: yupfvy date: 2022-11-14 05:02:14 failed to load
tid: yupick date: 2022-11-14 05:05:52 successfully loaded
tid: yupnjr date: 2022-11-14 05:13:39 s

tid: ywwygb date: 2022-11-16 15:46:23 successfully loaded
tid: ywxb05 date: 2022-11-16 15:58:35 successfully loaded
tid: ywzuzj date: 2022-11-16 17:24:06 successfully loaded
tid: yx0l2t date: 2022-11-16 17:49:01 successfully loaded
tid: yx0p3a date: 2022-11-16 17:52:59 failed to load
tid: yx0pjr date: 2022-11-16 17:53:28 failed to load
tid: yx12pu date: 2022-11-16 18:05:54 successfully loaded
tid: yx1pfe date: 2022-11-16 18:27:27 failed to load
tid: yx1wmq date: 2022-11-16 18:34:16 successfully loaded
tid: yx24e6 date: 2022-11-16 18:41:47 successfully loaded
tid: yx26ae date: 2022-11-16 18:43:40 failed to load
tid: yx351m date: 2022-11-16 19:17:01 failed to load
tid: yx3y8c date: 2022-11-16 19:45:19 successfully loaded
tid: yx62zu date: 2022-11-16 21:00:45 successfully loaded
tid: yx663e date: 2022-11-16 21:04:03 successfully loaded
tid: yx7ga6 date: 2022-11-16 21:53:30 failed to load
tid: yx7h7m date: 2022-11-16 21:54:32 successfully loaded
tid: yx7hl0 date: 2022-11-16 21:54:58 succes

tid: yzds0h date: 2022-11-19 15:00:08 failed to load
tid: yzedd4 date: 2022-11-19 15:26:38 successfully loaded
tid: yzg5pf date: 2022-11-19 16:44:50 successfully loaded
tid: yzh0gf date: 2022-11-19 17:22:00 failed to load
tid: yzhk3r date: 2022-11-19 17:45:42 successfully loaded
tid: yzhs49 date: 2022-11-19 17:55:34 successfully loaded
tid: yzhzqn date: 2022-11-19 18:04:19 successfully loaded
tid: yzib6k date: 2022-11-19 18:17:58 successfully loaded
tid: yzigwc date: 2022-11-19 18:25:01 failed to load
tid: yzjfl9 date: 2022-11-19 19:06:23 failed to load
tid: yzjl37 date: 2022-11-19 19:13:11 successfully loaded
tid: yzjy1x date: 2022-11-19 19:29:06 successfully loaded
tid: yzkb07 date: 2022-11-19 19:45:06 successfully loaded
tid: yzlcvu date: 2022-11-19 20:32:06 failed to load
tid: yzlg0g date: 2022-11-19 20:36:08 successfully loaded
tid: yzlgan date: 2022-11-19 20:36:31 successfully loaded
tid: yzmlf1 date: 2022-11-19 21:27:23 failed to load
tid: yzmyl0 date: 2022-11-19 21:43:58 succes

tid: z1yf4i date: 2022-11-22 16:48:41 successfully loaded
tid: z1yhgn date: 2022-11-22 16:51:17 failed to load
tid: z1z21f date: 2022-11-22 17:14:03 successfully loaded
tid: z1z8vl date: 2022-11-22 17:21:36 failed to load
tid: z21zpt date: 2022-11-22 19:09:22 successfully loaded
tid: z2218i date: 2022-11-22 19:11:00 successfully loaded
tid: z231oi date: 2022-11-22 19:50:12 successfully loaded
tid: z23kab date: 2022-11-22 20:09:49 successfully loaded
tid: z24irg date: 2022-11-22 20:47:12 failed to load
tid: z2540h date: 2022-11-22 21:09:27 successfully loaded
tid: z25uc1 date: 2022-11-22 21:36:29 successfully loaded
tid: z27576 date: 2022-11-22 22:25:56 successfully loaded
tid: z2794y date: 2022-11-22 22:30:09 successfully loaded
tid: z28322 date: 2022-11-22 23:01:34 successfully loaded
tid: z2aay0 date: 2022-11-23 00:34:03 successfully loaded
tid: z2ad1d date: 2022-11-23 00:36:45 successfully loaded
tid: z2ae7o date: 2022-11-23 00:38:18 failed to load
tid: z2b43d date: 2022-11-23 01:10

In [5]:
df = pd.read_csv('data/r_lakers_full_posts.csv', index_col=0, parse_dates=['datetime'])
df['month'] = df['datetime'].dt.month_name()
df_int = df[df['month'] == 'April']
df_int

,datetime,id,title,author,selftext,upvote_ratio,month
9963,2023-04-01 00:25:05,1285fd9,LeBron James Trash Talks Patrick Beverley &amp...,AboveHoops,NaN,1.0,April
9964,2023-04-01 00:36:05,1285q62,Bro does Bron even defend??,Dreni_Drozen,[removed],1.0,April
9965,2023-04-01 00:42:36,1285wch,Some stickers we made today,Johnzo_Ball,NaN,1.0,April
9966,2023-04-01 01:21:16,1286v93,Optimistic,bigmanrec,[removed],1.0,April
9967,2023-04-01 01:22:26,1286w86,Do you guys think we could trade Malik for kat,Funny_Statement3600,NaN,1.0,April
...,...,...,...,...,...,...,...
10994,2023-04-15 01:55:34,12mlvt7,We know what we need from you Darvin Hands. Ti...,CameraMan1,NaN,1.0,April
10995,2023-04-15 02:04:17,12mm482,“Ready to knock the lakers out of the playoffs”,jaltaffe,NaN,1.0,April
10996,2023-04-15 02:09:26,12mm8t1,Looks like Cancun has their #1 tourist on the ...,mrsupersumthing,NaN,1.0,April
10997,2023-04-15 02:38:16,12mn14z,"[Haynes] “DeMar DeRozan’s daughter, Diar, was ...",Alekesam1975,NaN,1.0,April


In [4]:
subreddit='lakers'
tid='1285fd9'
date='2023-04-01 00:25:05'
rr.get_all_comments(subreddit=subreddit,
                    tid=tid,
                    date=date)

AttributeError: 'DataFrame' object has no attribute 'id'

In [9]:
df_int = 

{'April', 'December', 'February', 'January', 'March', 'November', 'October'}